In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import pylatexenc


def compute_response(content):
    n = content['n']
    m = content['m']
    gates = content['gates']
    input_states = content['cats']
    output = content['output']
    
    circuit = Circuit(n, m, gates, input_states, output)
    return circuit.run()
    

def place_quantum_gate(gate, qc, row):
    
    if gate == 'x':
        qc.x(row)
    elif gate == 'y':
        qc.y(row)
    elif gate == 'z':
        qc.z(row)
    elif gate == 'h':
        qc.h(row)
    elif gate == 'i':
        qc.i(row)


def place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc):
    print('controlled: ', controlled_qubit)
    print('conditional: ', conditional_qubit)
    
    if gate == 'x':
        qc.cx(conditional_qubit, controlled_qubit)
    elif gate == 'y':
        qc.cy(conditional_qubit, controlled_qubit)
    elif gate == 'z':
        qc.cz(conditional_qubit, controlled_qubit)
    elif gate == 'h':
        qc.ch(conditional_qubit, controlled_qubit)
    elif gate == 'i':
        qc.ci(conditional_qubit, controlled_qubit)

        
def prepare_input_state(qc, qr, input_string):
    for i, bit in enumerate(input_string):
        if bit == '1':
            qc.x(qr[i])
    return qc
        

def handle_results(qc, qr, cr):
    qc.measure(qr, cr)

    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=1000)

    result = job.result()
    counts = result.get_counts(qc)

    print(counts)

    print(qc)
    qc.draw('mpl')
    
    return json.dumps(counts)
        

class Circuit:
    def __init__(self, n, m, gates_matrix, input_states, output_states):
        self.n = n
        self.m = m
        self.gates_matrix = gates_matrix
        self.input_states = input_states
        self.output_states = output_states

    def run(self):
        qr = QuantumRegister(self.n)
        cr = ClassicalRegister(self.n)
        qc = QuantumCircuit(qr, cr)
        
        prepare_input_state(qc, qr, self.input_states)

        for column in range(self.m):
            placed_gates = []
            column_gates = [row[column] for row in self.gates_matrix]
            controlled_gates = [gate_info for gate_info in column_gates if gate_info[0] == True]
            print(controlled_gates)
            
            for gate_info in controlled_gates:
                is_controlled, info = gate_info

                controlled_qubit, conditional_qubit, gate = info
                place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc)
                placed_gates.append(controlled_qubit)
                placed_gates.append(conditional_qubit)
            
            for row in range(self.n):
                gate_info = self.gates_matrix[row][column]
                
                if gate_info:
                
                    is_controlled, info = gate_info

                    if not is_controlled:
                        if row not in placed_gates:
                            place_quantum_gate(info, qc, row)

        return handle_results(qc, qr, cr)


In [ ]:
# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
import time

import json

hostName = "localhost"
serverPort = 8082

class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes(json.dumps(a, ensure_ascii=False), 'utf-8'))
    
    def do_POST(self):
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        
        content = json.loads(post_body)
        content_dict = json.loads(content)
        print('Request with body: ', content_dict)
        
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()

        message = compute_response(content_dict)
        self.wfile.write(bytes(message, "utf8"))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        passhttp://localhost:8888/notebooks/QuantumServer.ipynb#

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:8082
Request with body:  {'n': 2, 'm': 2, 'gates': [[[True, [0, 1, 'x']], [False, 'i']], [[False, 'i'], [True, [1, 0, 'x']]]], 'cats': '11', 'output': [0, 0]}
[[True, [0, 1, 'x']]]
controlled:  0
conditional:  1
[[True, [1, 0, 'x']]]
controlled:  1
conditional:  0


127.0.0.1 - - [30/Jun/2023 20:54:23] "POST / HTTP/1.1" 200 -


{'10': 1000}
      ┌───┐┌───┐     ┌─┐   
q0_0: ┤ X ├┤ X ├──■──┤M├───
      ├───┤└─┬─┘┌─┴─┐└╥┘┌─┐
q0_1: ┤ X ├──■──┤ X ├─╫─┤M├
      └───┘     └───┘ ║ └╥┘
c0: 2/════════════════╩══╩═
                      0  1 
Request with body:  {'n': 2, 'm': 2, 'gates': [[[False, 'i'], [True, [0, 1, 'x']]], [[True, [1, 0, 'x']], [False, 'i']]], 'cats': '11', 'output': [0, 0]}
[[True, [1, 0, 'x']]]
controlled:  1
conditional:  0
[[True, [0, 1, 'x']]]
controlled:  0
conditional:  1
{'01': 1000}
       ┌───┐     ┌───┐┌─┐   
q12_0: ┤ X ├──■──┤ X ├┤M├───
       ├───┤┌─┴─┐└─┬─┘└╥┘┌─┐
q12_1: ┤ X ├┤ X ├──■───╫─┤M├
       └───┘└───┘      ║ └╥┘
 c1: 2/════════════════╩══╩═
                       0  1 


127.0.0.1 - - [30/Jun/2023 20:54:56] "POST / HTTP/1.1" 200 -


Request with body:  {'n': 1, 'm': 1, 'gates': [[[False, 'i']]], 'cats': '1', 'output': [1]}
[]
{'1': 1000}
      ┌───┐┌───┐┌─┐
 q24: ┤ X ├┤ I ├┤M├
      └───┘└───┘└╥┘
c2: 1/═══════════╩═
                 0 


...

127.0.0.1 - - [30/Jun/2023 21:06:11] "POST / HTTP/1.1" 200 -


Request with body:  {'n': 1, 'm': 1, 'gates': [[[False, 'y']]], 'cats': '0', 'output': [1]}
[]
{'1': 1000}
       ┌───┐┌─┐
 q252: ┤ Y ├┤M├
       └───┘└╥┘
c23: 1/══════╩═
             0 


127.0.0.1 - - [30/Jun/2023 21:06:23] "POST / HTTP/1.1" 200 -


Request with body:  {'n': 1, 'm': 1, 'gates': [[[False, 'y']]], 'cats': '1', 'output': [0]}
[]
{'0': 1000}
       ┌───┐┌───┐┌─┐
 q261: ┤ X ├┤ Y ├┤M├
       └───┘└───┘└╥┘
c24: 1/═══════════╩═
                  0 


127.0.0.1 - - [30/Jun/2023 21:06:33] "POST / HTTP/1.1" 200 -
